In [30]:
import torch
import torch.nn as nn
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from skimage import data, img_as_float
from skimage.segmentation import chan_vese
import os
import cv2
import numpy as np

def cal_visibility_based_on_chan_vese(I, label):
    """calculate visibility within each box, namely the ratio of foreground pixels in the box
    
    Args:
        I (_type_): original gray image with shape of (H, W)
        x (_type_): x centre
        y (_type_): y centre
        width (_type_): width
        height (_type_): height
    """
    image_width, image_height = I.shape
    
    x, y, width, height = label[0], label[1], label[2], label[3]
    target = I[max(0, int((y - height / 2) * image_width)) : min(image_height, int((y + height / 2) * image_width)), max(0, int((x - width / 2) * image_height)) : min(image_width, int((x + width / 2) * image_height))]
    cv = chan_vese(target, mu=0.05, lambda1=1, lambda2=1, tol=1e-3, max_num_iter=200,
                dt=0.5, init_level_set="checkerboard", extended_output=True)

    seg_mask = cv[0]
    
    # change cordinate of seg_mask to the original image
    y1, y2, x1, x2 = max(0, int((y - height / 2) * image_width)), min(image_height, int((y + height / 2) * image_width)), max(0, int((x - width / 2) * image_height)), min(image_width, int((x + width / 2) * image_height))
    seg_mask_original = np.zeros_like(I)
    seg_mask_original[y1:y2, x1:x2] = seg_mask
    
    
    x_bar = np.sum(target[seg_mask == True]) / np.prod(target.shape)
    
    seg_mask_original_inverse = np.ones_like(seg_mask_original) - seg_mask_original
    background_extended = (I * seg_mask_original_inverse)[max(0, int((y - height) * image_width)) : min(image_height, int((y + height) * image_width)), max(0, int((x - width) * image_height)) : min(image_width, int((x + width) * image_height))]
    
    
    
    x_bar_ = np.sum(background_extended) / np.prod(background_extended.shape)
    
    v_a = np.abs(x_bar - x_bar_) / np.max(I)

    return v_a


def cal_distribution_visibility(vis):
    """

    Args:
        vis (np.array): one-dimensioned vector of visibility of all defects over dataset
    Returns:
        _type_: _description_
    """
    return np.var(vis, keepdims=True, dtype=np.float32)



def cal_exposure(I, threshold:int=245):
    """calculate exposure
    Args:
        I (_type_): original gray image with shape of (H, W)
        threshold (_type_): threshold
    """
    return np.sum(I >= threshold) / np.prod(I.shape)


def cal_overexposure_and_overdarkness(I, high_threshold:int=240, low_threshold:int=20):
    """calculate overexposure and overdarkness
    Args:
        I (_type_): original gray image with shape of (H, W)
        high_threshold (_type_): high threshold
        low_threshold (_type_): low threshold
    """
    return (np.sum(I >= high_threshold) + np.sum(I <= low_threshold)) / np.prod(I.shape)



def cal_comprehensive_score(expo, distri, vis, alpha=1/3, beta=1/3, gama=1/3):
    """calculate comprehensive score
    Args:
        expo (_type_): exposure
        distri (_type_): distribution visibility
        vis (_type_): visibility
    """
    return alpha * vis + beta * (1 - distri) + gama * (1 - expo)


def quality_prediction_of_dataset(path):
    """quality prediction of dataset
    Args:
        path (_type_): path of dataset
    """
    visibility_record = []
    exposure_record = []
    for file in os.listdir(os.path.join(path, "images")):
        prelix = file[:-4]
        image_path = os.path.join(path, "images", file)
        label_path = os.path.join(path, "labels", prelix+".txt")
        
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        labels = np.loadtxt(label_path)
        
        if len(labels.shape) == 2:
            for label in labels:        
                visibility = cal_visibility_based_on_chan_vese(img, label[1:])
                visibility_record.append(visibility)
        else:
            visibility = cal_visibility_based_on_chan_vese(img, labels[1:])
            visibility_record.append(visibility)
                    
        # exposure = cal_exposure(img)
        exposure = cal_overexposure_and_overdarkness(img)
        exposure_record.append(exposure)
    
    visibility = np.mean(np.array(visibility_record), axis=0)
    exposure = np.mean(np.array(exposure_record), axis=0)
    distribution_visibility = np.var(np.array(visibility_record / np.max(visibility_record)), axis=0)
    
    score = cal_comprehensive_score(exposure, distribution_visibility, visibility)
    

    return score, visibility, exposure, distribution_visibility


# import pandas

# columns = ['dataset', 'score', 'visibility', 'exposure', 'distribution_visibility']
# for i in [70]:
#     # save to csv
#     score, vis, expo, distr = quality_prediction_of_dataset(f'/Data4/student_zhihan_data/data/GC10-DET_brightness_{i}/valid')
#     print(['GC10-DET_brightness_'+str(i), score, vis, expo, distr])
#     df = pandas.DataFrame([['GC10-DET_brightness_'+str(i), score, vis, expo, distr]], columns=columns)
#     df.to_csv('demo.csv', mode='a', header=False, index=False)
    

In [5]:

valid_re = [['GC10-DET_brightness_-150', 0.4396265921451846, 0.0814905927558818, 0.7469545170298794, 0.01565629929044857],
['GC10-DET_brightness_-100', 0.5765332267984907, 0.16759716235507985, 0.3962311487030564, 0.041766333256551255],
['GC10-DET_brightness_-50', 0.6598785597036272, 0.23041798135195282, 0.20241752758360745, 0.04836477465746381],
['GC10-DET_brightness_-30', 0.6832842647650346, 0.2423731897761544, 0.14062478584155705, 0.0518956096394935],
['GC10-DET_brightness_-25', 0.6896770602342559, 0.2451073532863147, 0.12367873877809758, 0.052397433805449624],
['GC10-DET_brightness_-20', 0.6963008446511398, 0.24748917231865877, 0.10584245112904331, 0.05274418723619604],
['GC10-DET_brightness_-15', 0.7037670019351863, 0.2501006258301439, 0.08599790338884321, 0.05280171663574162],
['GC10-DET_brightness_-10', 0.7116716641119891, 0.2522334657952452, 0.06434874417489035, 0.052869729284387254],
['GC10-DET_brightness_10', 0.7372806075717755, 0.27499172275325684, 0.013928452876576207, 0.04922144716135431],
['GC10-DET_brightness_20', 0.7386991959316531, 0.29094555854127885, 0.025586719178316886, 0.04926125156800252],
['GC10-DET_brightness_30', 0.7359570300691421, 0.30771670563578113, 0.04892284727933113, 0.05092276814902331],
['GC10-DET_brightness_50', 0.7184395474444605, 0.34236278720162244, 0.13174868935032896, 0.055295455517912026],
['GC10-DET_brightness_60', 0.7031320737743537, 0.36200593357097643, 0.1914946064196135, 0.0611151058283016],
['GC10-DET_brightness_70', 0.687603538472578, 0.38757405664470773, 0.2604017023454633, 0.06436173888151044],
['GC10-DET_brightness_90', 0.6531932842052922, 0.4444649873654304, 0.41056899221319904, 0.07431614253635478],
['GC10-DET_brightness_0', 0.7276038178563639, 0.26117753587034265, 0.027682677151864035, 0.05068340514938684]
]

In [6]:
re = [['GC10-DET_brightness_-150', 0.42715077810322377, 0.07020707070055898, 0.7661689453125, 0.02258579107838751],
['GC10-DET_brightness_-100', 0.5629529207144853, 0.15643331987060077, 0.4232231869904891, 0.04435137073665557],
['GC10-DET_brightness_-50', 0.6519661736607636, 0.21161311368918934, 0.20523657226562503, 0.050478020441273155],
['GC10-DET_brightness_-30', 0.6810303218384329, 0.22497848038382703, 0.13085241168478262, 0.05103510318374548],
['GC10-DET_brightness_-25', 0.6879250307331253, 0.22705775365920466, 0.1122604449728261, 0.05102221648700254],
['GC10-DET_brightness_-20', 0.6947384709260322, 0.22983390396385378, 0.09358351732336957, 0.05203497386238705],
['GC10-DET_brightness_-15', 0.7017444179194128, 0.23254492005214036, 0.07545116126019022, 0.051860505033711854],
['GC10-DET_brightness_-10', 0.7082613901355719, 0.23510042870806452, 0.058794104534646736, 0.051522153766702085],
['GC10-DET_brightness_30', 0.7331852046115299, 0.2919299063305358, 0.046353186565896744, 0.04602110593004898],
['GC10-DET_brightness_60', 0.7048761530046108, 0.3448507602205629, 0.1774439538043478, 0.05277834740238259],
['GC10-DET_brightness_90', 0.6566731784203799, 0.42314135055772095, 0.38285598887567934, 0.07026582642090194],
['GC10-DET_brightness_50', 0.7185677762933553, 0.3280458604819985, 0.12302671747622283, 0.04931581412570976],
['GC10-DET_brightness_70', 0.6892318419112333, 0.366129408086266, 0.24061327063519022, 0.0578206117173759],
['GC10-DET_brightness_110', 0.6223764578372124, 0.4676211901381175, 0.5211276749320652, 0.07936414169441483],
['GC10-DET_brightness_0', 0.7218066864345708, 0.24574374061695708, 0.028097486413043476, 0.052226194900201105],
['GC10-DET_brightness_10', 0.731236667602783, 0.2584208464158037, 0.01353161090353261, 0.05117923270392171],
['GC10-DET_brightness_20', 0.7334215143229427, 0.27302691538001994, 0.024355288298233694, 0.048407084112957896]
]

In [31]:
# # # add to excel
# import pandas as pd
# df = pd.DataFrame(re, columns=['dataset', 'score', 'visibility', 'exposure', 'distribution_visibility'])
# df.to_excel('test_details.xlsx', index=False)

In [32]:
# df

,dataset,score,visibility,exposure,distribution_visibility
0,GC10-DET_brightness_-150,0.427151,0.070207,0.766169,0.022586
1,GC10-DET_brightness_-100,0.562953,0.156433,0.423223,0.044351
2,GC10-DET_brightness_-50,0.651966,0.211613,0.205237,0.050478
3,GC10-DET_brightness_-30,0.681030,0.224978,0.130852,0.051035
4,GC10-DET_brightness_-25,0.687925,0.227058,0.112260,0.051022
5,GC10-DET_brightness_-20,0.694738,0.229834,0.093584,0.052035
6,GC10-DET_brightness_-15,0.701744,0.232545,0.075451,0.051861
7,GC10-DET_brightness_-10,0.708261,0.235100,0.058794,0.051522
8,GC10-DET_brightness_30,0.733185,0.291930,0.046353,0.046021
9,GC10-DET_brightness_60,0.704876,0.344851,0.177444,0.052778


In [50]:
# read xlsx file
import pandas as pd

# Read the Excel file
data = pd.read_excel('/Data4/student_zhihan_data/source_code/Mydemo/results.xlsx', sheet_name='val_1')
# data = df[df['Unnamed: 0'].isin([-100,-50, -30, 0, 30, 60, 90])]


['GC10-DET_brightness_10', 0.731236667602783, 0.2584208464158037, 0.01353161090353261, 0.05117923270392171]
['GC10-DET_brightness_20', 0.7334215143229427, 0.27302691538001994, 0.024355288298233694, 0.048407084112957896]
['GC10-DET_brightness_50', 0.7185677762933553, 0.3280458604819985, 0.12302671747622283, 0.04931581412570976]

In [51]:
data

,light,p,r,mAP50,mAP50:95,f1,f2,Proposed,NIQE_train,NIQE_self_train,BRISQUE,NIQE,vis,exposure,distribution
0,-150,0.480,0.393,0.386,0.164,0.432165,0.407782,0.4396,18.6022,6.6069,43.2107,13.7729,0.081491,0.746955,0.015656
1,-100,0.595,0.580,0.600,0.288,0.587404,0.582939,0.5765,17.4120,8.4795,42.4839,14.8153,0.167597,0.396231,0.041766
2,-50,0.664,0.570,0.604,0.293,0.613420,0.586609,0.6599,7.2026,5.4698,40.8140,16.1725,0.230418,0.202418,0.048365
3,-30,0.581,0.639,0.614,0.304,0.608621,0.626492,0.6833,2.8972,2.7279,37.2681,16.7846,0.242373,0.140625,0.051896
4,-20,0.612,0.558,0.598,0.296,0.583754,0.568024,0.6963,2.6986,2.6450,34.9551,17.4660,0.247489,0.105842,0.052744
5,-15,0.642,0.578,0.598,0.297,0.608321,0.589758,0.7038,2.6594,2.6420,34.4868,17.7351,0.250101,0.085998,0.052802
6,-10,0.626,0.582,0.601,0.294,0.603199,0.590298,0.7117,2.6449,2.6481,34.6845,17.9540,0.252233,0.064349,0.052870
7,0,0.645,0.597,0.622,0.310,0.620072,0.606020,0.7276,2.6496,2.6496,34.7385,17.9345,0.261178,0.027683,0.050683
8,10,0.557,0.620,0.596,0.290,0.586814,0.606285,0.7373,2.6487,2.6530,34.7075,17.7401,0.274992,0.013928,0.049221
9,20,0.569,0.572,0.579,0.279,0.570496,0.571397,0.7387,2.6761,2.6623,34.7110,17.1023,0.290946,0.025587,0.049261


In [52]:
# write a function for find the best coefficient weights of three parameters (vis, expo, distr) in order to get the best score close to ground truth
def find_best_parameter(): 
    # read xlsx file to get ground truth
    ground_truth = []
    vis_score = []
    expo_score = []
    distr_score = []
    
    configures = data['light']
    for i in configures:
        dataset_name = 'GC10-DET_brightness_' + str(i)
        for j in valid_re:
            if j[0] == dataset_name:
                vis_score.append(j[2])
                expo_score.append(j[3])
                distr_score.append(j[4])
                ground_truth.append(data[data['light'] == i]['f2'].values[0])
                break
        
    vis_score = np.array(vis_score)
    expo_score = np.array(expo_score)
    distr_score = np.array(distr_score)
    ground_truth = np.array(ground_truth)
    
    grid_space = []
    mse_errors = []
    mape_errors = []
    srcc_errors = []
    
    
    # grid searching
    for vis in np.arange(0, 1, 0.01):
        for expo in np.arange(0, 1, 0.01):
            for distr in np.arange(0, 1, 0.01):
                if vis + expo + distr != 1:
                    continue
               
                grid_space.append([vis, expo, distr])
                                
                score_predicted = vis * vis_score + expo * (1 - expo_score) + distr * (1 - distr_score)
                # caculate mse error
                error = np.mean(np.square(score_predicted - ground_truth))
                mse_errors.append(error)
                
                # caculate mape error
                error = np.mean(np.abs((score_predicted - ground_truth) / ground_truth))
                mape_errors.append(error)
                
                # caculate max SRCC
                error = np.corrcoef(score_predicted, ground_truth)[0, 1]
                srcc_errors.append(error)
    
    return grid_space, mse_errors, mape_errors, srcc_errors

In [53]:
grid_space, mse_errors, mape_errors, srcc_errors = find_best_parameter()

In [54]:
ground_truth = []
vis_score = []
expo_score = []
distr_score = []

configures = data['light']
for i in configures:
    dataset_name = 'GC10-DET_brightness_' + str(i)
    for j in re:
        if j[0] == dataset_name:
            vis_score.append(j[2])
            expo_score.append(j[3])
            distr_score.append(j[4])
            ground_truth.append(data[data['light'] == i]['f2'].values[0])
            break
    
vis_score = np.array(vis_score)
expo_score = np.array(expo_score)
distr_score = np.array(distr_score)
ground_truth = np.array(ground_truth)

In [55]:
ground_truth

array([0.4077821 , 0.58293919, 0.5866088 , 0.62649173, 0.56802395,
       0.58975842, 0.59029812, 0.60601983, 0.60628511, 0.57139747,
       0.58055556, 0.60421399, 0.60175342, 0.51425034, 0.58303831])

In [56]:
data

,light,p,r,mAP50,mAP50:95,f1,f2,Proposed,NIQE_train,NIQE_self_train,BRISQUE,NIQE,vis,exposure,distribution
0,-150,0.480,0.393,0.386,0.164,0.432165,0.407782,0.4396,18.6022,6.6069,43.2107,13.7729,0.081491,0.746955,0.015656
1,-100,0.595,0.580,0.600,0.288,0.587404,0.582939,0.5765,17.4120,8.4795,42.4839,14.8153,0.167597,0.396231,0.041766
2,-50,0.664,0.570,0.604,0.293,0.613420,0.586609,0.6599,7.2026,5.4698,40.8140,16.1725,0.230418,0.202418,0.048365
3,-30,0.581,0.639,0.614,0.304,0.608621,0.626492,0.6833,2.8972,2.7279,37.2681,16.7846,0.242373,0.140625,0.051896
4,-20,0.612,0.558,0.598,0.296,0.583754,0.568024,0.6963,2.6986,2.6450,34.9551,17.4660,0.247489,0.105842,0.052744
5,-15,0.642,0.578,0.598,0.297,0.608321,0.589758,0.7038,2.6594,2.6420,34.4868,17.7351,0.250101,0.085998,0.052802
6,-10,0.626,0.582,0.601,0.294,0.603199,0.590298,0.7117,2.6449,2.6481,34.6845,17.9540,0.252233,0.064349,0.052870
7,0,0.645,0.597,0.622,0.310,0.620072,0.606020,0.7276,2.6496,2.6496,34.7385,17.9345,0.261178,0.027683,0.050683
8,10,0.557,0.620,0.596,0.290,0.586814,0.606285,0.7373,2.6487,2.6530,34.7075,17.7401,0.274992,0.013928,0.049221
9,20,0.569,0.572,0.579,0.279,0.570496,0.571397,0.7387,2.6761,2.6623,34.7110,17.1023,0.290946,0.025587,0.049261


In [57]:
# find min index
min_index = np.argmax(srcc_errors)
second_min_index = np.argmax(srcc_errors[:min_index])

# print best parameter
best_parameter = grid_space[min_index]
print(best_parameter)
print(mse_errors[min_index])

# best_parameter = grid_space[second_min_index]
# print(best_parameter)
# print(mse_errors[second_min_index])

score_predicted = best_parameter[0] * vis_score + best_parameter[1] * (1 - expo_score) + best_parameter[2] * (1 - distr_score)
print(score_predicted)

from scipy import stats
ground_truth_rank = stats.rankdata(ground_truth)
score_predicted_rank = stats.rankdata(score_predicted)
print(stats.spearmanr(ground_truth_rank, score_predicted_rank))
print(stats.pearsonr(ground_truth, score_predicted))

[0.36, 0.64, 0.0]
0.012157993675106189
[0.17492642 0.42545316 0.58482931 0.63724671 0.66284675 0.67542743
 0.68700793 0.71048536 0.72437127 0.72270231 0.71542873 0.67935941
 0.65058214 0.61781409 0.54730305]
SignificanceResult(statistic=0.37499999999999994, pvalue=0.16843274984674558)
PearsonRResult(statistic=0.7797018346705482, pvalue=0.0006086156165833346)


In [46]:
# find min index
min_index = np.argmin(mape_errors)

# print best parameter
best_parameter = grid_space[min_index]
print(best_parameter)
print(mse_errors[min_index])

score_predicted = best_parameter[0] * vis_score + best_parameter[1] * (1 - expo_score) + best_parameter[2] * (1 - distr_score)
print(score_predicted)

from scipy import stats
ground_truth_rank = stats.rankdata(ground_truth)
score_predicted_rank = stats.rankdata(score_predicted)
print(stats.spearmanr(ground_truth_rank, score_predicted_rank))
print(stats.pearsonr(ground_truth, score_predicted))

[0.52, 0.15, 0.33]
0.0019591369047575993
[0.39412902 0.4832259  0.54259559 0.56051936 0.56830456 0.57249172
 0.5764308  0.58633748 0.59545995 0.60234636 0.60966361 0.61585562
 0.61528895 0.6152145  0.61941738]
SignificanceResult(statistic=0.1464285714285714, pvalue=0.602549920983054)
PearsonRResult(statistic=0.6651955145273444, pvalue=0.006806806482865556)


In [47]:
ground_truth

array([0.4077821 , 0.58293919, 0.5866088 , 0.62649173, 0.56802395,
       0.58975842, 0.59029812, 0.60601983, 0.60628511, 0.57139747,
       0.58055556, 0.60421399, 0.60175342, 0.51425034, 0.58303831])

In [48]:
score_predicted = 1/3 * vis_score + 1/3 * (1 - expo_score) + 1/3 * (1 - distr_score)

In [49]:
from scipy import stats
ground_truth_rank = stats.rankdata(ground_truth)
score_predicted_rank = stats.rankdata(score_predicted)
print(stats.spearmanr(ground_truth_rank, score_predicted_rank))
print(stats.pearsonr(ground_truth, score_predicted))

SignificanceResult(statistic=0.3, pvalue=0.27731678036838386)
PearsonRResult(statistic=0.7664665860345516, pvalue=0.0008590179330831546)
